<a href="https://colab.research.google.com/github/KudCh/BAINFOR17_FRP_Lab_My_Copy/blob/master/ukraine_news_bias_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchtext.models import T5Transform

padding_idx = 0
eos_idx = 1
max_seq_len = 512
t5_sp_model_path = "https://download.pytorch.org/models/text/t5_tokenizer_base.model"

transform = T5Transform(
    sp_model_path=t5_sp_model_path,
    max_seq_len=max_seq_len,
    eos_idx=eos_idx,
    padding_idx=padding_idx,
)

In [ ]:
from torchtext.models import T5_BASE_GENERATION
import torch


t5_base = T5_BASE_GENERATION
transform = t5_base.transform()
model = t5_base.get_model()
#model.cuda()
model.eval()

T5Model(
  (token_embeddings): Embedding(32128, 768, padding_idx=0)
  (encoder): T5Encoder(
    (token_embeddings): Embedding(32128, 768, padding_idx=0)
    (layers): ModuleList(
      (0): T5Layer(
        (self_attn): T5MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=False)
          (relative_attention_bias): Embedding(32, 12)
        )
        (linear1): Linear(in_features=768, out_features=3072, bias=False)
        (linear2): Linear(in_features=3072, out_features=768, bias=False)
        (norm1): T5LayerNorm()
        (norm2): T5LayerNorm()
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
        (dropout3): Dropout(p=0.0, inplace=False)
      )
      (1-11): 11 x T5Layer(
        (self_attn): T5MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=False)
        )
        (linear1): Linear(in_features=768,

In [ ]:
from torchtext.prototype.generate import GenerationUtils

sequence_generator = GenerationUtils(model)

# Preprocessing
We wanna ~~create a datapipe~~ preprocess the dataset, create a custom Pytorch dataset and then create a dataloader from it

In [ ]:
dataset_name = "average"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# load data
data = pd.read_csv("/content/drive/MyDrive/ukraine-news-bias-dataset/subjectivity_{}.csv".format(dataset_name))

# 2 classes:
# 0 - objective, 1 - subjective
# average: < 1, >= 1
# majority: < 0.5, >= 0.5
# intensified: < 0.5, >= 0.5
limit = None
if dataset_name == "average":
    limit = 1.0
else:
    limit = 0.5
data["labels"] = np.where(data["scores"] < limit, 0, 1)

# --- preparing data for training---
X = pd.DataFrame(data.sentences)
y = pd.DataFrame(data.labels)

# set aside 20% of train and test data for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, shuffle=True,
                                                    random_state=8)  # 20% for testing and validation

data = data[["sentences", "labels"]]
train, test = train_test_split(data, test_size=0.2)
validation, test = train_test_split(test, test_size=0.5)

In [ ]:
#prepend prefixes to data
prefix = ["binary classification" for i in range(len(data))]
data["prefix"]=prefix
df = data.rename(columns={"sentences": "input_text", "labels": "target_text"})

train, test = train_test_split(df, test_size=0.2)
validation, test = train_test_split(test, test_size=0.5)

In [ ]:
!pip install 'portalocker>=2.0.0'
import portalocker
from functools import partial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchdata.datapipes.iter import IterableWrapper

class MyDataset(Dataset):

    def apply_prefix(task, x):
        return f"{task}: " + x

    def label_to_text(label):
        return str(label)

    def __init__(self, data, task, transform=apply_prefix, target_transform=label_to_text):
        #self.img_labels = pd.read_csv(annotations_file)
        #self.img_dir = img_dir
        self.sentences = data["input_text"].values
        self.labels = data["target_text"].values
        self.transform = transform
        self.target_transform = target_transform
        self.task = task

    def __len__(self):
        return len(self.labels)
      
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]

        if self.transform:
            sentence = self.transform(self.task, sentence)
        if self.target_transform:
            label = self.target_transform(label)
        pipe = {"sentence":sentence, "label":label}
        return pipe

In [ ]:
training_data = MyDataset(train, task="binary classification")
# apply ToTensor() as transform

validation_data = MyDataset(validation, task="binary classification")
test_data = MyDataset(test, task="binary classification")

train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=5, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

In [ ]:
batch = next(iter(train_dataloader))
input_text = batch["sentence"]
target = batch["label"]
beam_size = 1
data_batch_size = 5

model_input = transform(input_text)
model_output = sequence_generator.generate(model_input, eos_idx=eos_idx, num_beams=beam_size)
output_text = transform.decode(model_output.tolist())

for i in range(data_batch_size):
    print(f"Example {i+1}:\n")
    print(f"input_text: {input_text[i]}\n")
    print(f"prediction: {output_text[i]}\n")
    print(f"target: {target[i]}\n\n")

Example 1:

input_text: binary classification: They include use of force, economic and propaganda pressure, meddling in domestic affairs, and appeals to a kind of ‘supra-legal’ legitimacy when they need to justify illegal intervention in this or that conflict or toppling inconvenient regimes.

prediction: a number of reasons for using force include meddling in domestic affairs, economic and propaganda pressure, meddling in this or that conflict or toppling inconvenient regimes

target: 0


Example 2:

input_text: binary classification: Yet caution should be exercised in using this number, because all active members do not belong to units that have trained together, although their commitment to defend their country can be assumed to be high.

prediction: - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 mill

In [ ]:
f = open("/content/drive/MyDrive/ukraine-news-bias-dataset/outputs.txt", "w")
f.write('target')
for i in target:
    f.write(i+'\n')
f.write('output')
for i in output_text:
    f.write(i)
f.close()

In [ ]:
print(target)
print(output_text)

['0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['a number of reasons for using force include meddling in domestic affairs, economic and propaganda pressure, meddling in this or that conflict or toppling inconvenient regimes', '- 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million - 1 million ', '', 'False', 'False', '', 'False', 'a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone wolf - a lone', 'False', '- Unknown to me, but I am not sure if this is a good thing.', '- 10:55 - Russia\'s Foreign Ministry has called on Western powers to give up their "destructive" policy on Ukraine